In [56]:
import pandas as pd
import numpy as np

In [57]:
data = pd.read_csv("./data/train_dataset.csv")

data['is_test'] = False
evaluation = pd.read_csv("./data/evaluation_public.csv")
evaluation['is_test'] = True
sample = pd.read_csv("./data/sample_submission.csv")

all_data = pd.concat([data, evaluation]).reset_index(drop=True)

# 特征处理

In [58]:
all_data['date'] = pd.to_datetime(all_data['time'])
all_data['hour'] = all_data['date'].dt.hour
all_data['year'] = all_data['date'].dt.year
all_data['month'] = all_data['date'].dt.month
all_data['minute'] = all_data['date'].dt.minute
all_data['weekday'] = all_data['date'].dt.weekday
all_data['day'] = all_data['date'].dt.day
all_data['hour'] = all_data['date'].dt.hour
all_data['ts'] = all_data['hour']*3600 + all_data['minute']*60 + all_data['date'].dt.second

In [59]:
import gc
features = [ 'JS_NH3', 'CS_NH3', 'JS_TN', 'CS_TN', 'JS_LL', 'CS_LL', 'MCCS_NH4', 'MCCS_NO3', 'JS_COD', 'CS_COD', 'JS_SW', 'CS_SW', 'B_HYC_NH4', 'B_HYC_XD', 'B_HYC_MLSS', 'B_HYC_JS_DO', 'B_HYC_DO', 'B_CS_MQ_SSLL', 'B_QY_ORP', 'N_HYC_NH4', 'N_HYC_XD', 'N_HYC_MLSS', 'N_HYC_JS_DO', 'N_HYC_DO', 'N_CS_MQ_SSLL', 'N_QY_ORP','weekday','hour', 'ts']

# N_HYC_JS_DO
filter_set = {'time', 'Label1', 'Label2','CS_LL','CS_NH3', 'JS_SW', 'B_QY_ORP'}
features = [f for f in features if f not in filter_set] #
for f in features:
    all_data[f] = all_data[f].fillna(method='ffill')

labels = ['Label1', 'Label2']
train = all_data[~all_data['is_test']].copy(deep=True)
test = all_data[all_data['is_test']].copy(deep=True)
# train = train.dropna(subset=['Label1', 'Label2']).reset_index(drop=True)
test['is_train'] = False
train['is_train'] = True
data = pd.concat([train, test]).reset_index(drop=True)

del all_data,train, test
gc.collect()

26

# 通过训练找出在训练集和测试集中差别很大的样本

In [60]:
from sklearn.model_selection import train_test_split
for f in features:
    if f not in ['weekday','hour', 'ts']:
        q = len(data[f].drop_duplicates())
        data[f] = pd.qcut(data[f], q=int(q/500), labels=False, duplicates="drop")
data['is_test'] = data['is_test'].apply(lambda x: 1 if x  else 0)

In [61]:
data.describe()

,JS_NH3,CS_NH3,JS_TN,CS_TN,JS_LL,CS_LL,MCCS_NH4,MCCS_NO3,JS_COD,CS_COD,...,Label1,Label2,is_test,hour,year,month,minute,weekday,day,ts
count,150480.000000,123432.000000,150480.000000,150480.000000,150480.000000,130970.000000,150480.000000,150480.000000,150480.000000,0.0,...,35068.000000,35068.000000,150480.000000,150480.00000,150480.0,150480.000000,150480.00000,150480.000000,150480.000000,150480.000000
mean,3.430323,0.041165,2.998212,2.931918,114.172488,1249.096494,1.499415,4.959204,1.998664,NaN,...,13745.474316,11816.071793,0.066454,11.50000,2022.0,4.062201,29.00000,3.014354,15.856459,43140.000000
std,2.230066,0.065458,2.000081,2.080676,64.459570,873.108749,1.117776,3.130447,1.414680,NaN,...,4717.251176,3664.571351,0.249075,6.92221,0.0,1.985831,17.31094,1.993965,8.674275,24941.590446
min,0.000000,0.001100,0.000000,0.000000,0.000000,-3447.026900,0.000000,0.000000,0.000000,NaN,...,814.490000,719.710000,0.000000,0.00000,2022.0,1.000000,0.00000,0.000000,1.000000,0.000000
25%,1.000000,0.013300,1.000000,1.000000,64.000000,849.855700,0.000000,2.000000,1.000000,NaN,...,10498.402500,9269.437500,0.000000,5.75000,2022.0,2.000000,14.00000,1.000000,8.000000,21570.000000
50%,3.000000,0.035100,3.000000,3.000000,106.500000,1326.369300,1.000000,5.000000,2.000000,NaN,...,13071.830000,11159.960000,0.000000,11.50000,2022.0,4.000000,29.00000,3.000000,16.000000,43140.000000
75%,5.000000,0.056500,5.000000,5.000000,168.000000,1801.756300,2.000000,7.000000,3.000000,NaN,...,15359.617500,13203.822500,0.000000,17.25000,2022.0,6.000000,44.00000,5.000000,23.000000,64710.000000
max,7.000000,1.495900,6.000000,6.000000,232.000000,3414.167000,3.000000,10.000000,4.000000,NaN,...,66326.050000,49464.130000,1.000000,23.00000,2022.0,7.000000,58.00000,6.000000,31.000000,86280.000000


In [65]:
import lightgbm as lgb
use_features = features
filter_set = {'time', 'Label1', 'Label2','CS_LL','CS_NH3', 'JS_SW', 'B_QY_ORP', 'CS_SW',
              'CS_COD'}
use_features = [f for f in use_features if f not in filter_set] #
X_train, X_test, y_train, y_test  = train_test_split(data[use_features], data['is_test'], test_size=0.2)
model = lgb.LGBMClassifier(
        boosting="gbdt",
        max_depth=4,
        learning_rate=0.005,
        n_estimators=500,
        min_child_weight=1,
        min_data_in_leaf=60,
        subsample = 0.7,
        feature_fraction=0.4,
        bagging_seed=1,
        reg_alpha=0.11,
        reg_lambda=0.1,  # 此处不改了
        min_sum_hessian_in_leaf=0.01,
        random_state=1212
    )
model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], eval_metric=['auc'],
          early_stopping_rounds=20, verbose=200)
df = pd.DataFrame()
df['score'] = model.feature_importances_
df['features']= use_features
df.sort_values(by=['score'], ascending=False, inplace=True, ignore_index=True)

Training until validation scores don't improve for 20 rounds
[200]	training's auc: 0.999313	training's binary_logloss: 0.0917317	valid_1's auc: 0.999286	valid_1's binary_logloss: 0.0913485
[400]	training's auc: 0.99961	training's binary_logloss: 0.0516857	valid_1's auc: 0.999601	valid_1's binary_logloss: 0.0515993
Did not meet early stopping. Best iteration is:
[500]	training's auc: 0.999695	training's binary_logloss: 0.0411222	valid_1's auc: 0.999696	valid_1's binary_logloss: 0.0410786


In [66]:
df

,score,features
0,609,N_HYC_JS_DO
1,577,B_HYC_JS_DO
2,564,MCCS_NO3
3,542,MCCS_NH4
4,518,N_HYC_XD
5,496,B_HYC_XD
6,439,N_HYC_MLSS
7,369,weekday
8,344,JS_TN
9,310,B_HYC_MLSS


In [64]:
# import lightgbm as lgb
# features_list = [ 'JS_NH3', 'CS_NH3', 'JS_TN', 'CS_TN', 'JS_LL', 'MCCS_NH4', 'MCCS_NO3', 'JS_COD', 'CS_COD', 'JS_SW', 'CS_SW', 'B_HYC_NH4', 'B_HYC_XD', 'B_HYC_MLSS', 'B_HYC_JS_DO', 'B_HYC_DO', 'B_CS_MQ_SSLL', 'B_QY_ORP', 'N_HYC_NH4', 'N_HYC_XD', 'N_HYC_MLSS', 'N_HYC_JS_DO', 'N_HYC_DO', 'N_CS_MQ_SSLL']
#
# feature_score_dict = {}
# for i in range(len(features_list)-1):
#     for j in range(i+1, len(features_list)):
#         features_filter = ['ts', 'weekday', features_list[i], features_list[j]]
#         X_train, X_test, y_train, y_test  = train_test_split(data_v2[features_filter], data_v2['is_test'], test_size=0.2)
#         model = lgb.LGBMClassifier(
#                 boosting="gbdt",
#                 max_depth=4,
#                 learning_rate=0.005,
#                 n_estimators=500,
#                 min_child_weight=1,
#                 min_data_in_leaf=60,
#                 subsample = 0.7,
#                 feature_fraction=0.4,
#                 bagging_seed=1,
#                 reg_alpha=0.11,
#                 reg_lambda=0.1,  # 此处不改了
#                 min_sum_hessian_in_leaf=0.01,
#                 random_state=1212
#             )
#         model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], eval_metric=['auc'],
#                   early_stopping_rounds=20, verbose=200)
#         feature_score_dict[f'{features_list[i]}_{features_list[j]}_score'] = model.best_score_['valid_1']['auc']
#
